In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import json
import os
from scipy.stats import mstats

# Data Loading and Manipulation

In [3]:
data = pd.read_csv("/Users/beneverman/Documents/Coding/QuantHive/IDVF-Oxford-v1/data/processed-5yr-93-minute/65min.csv", index_col=0)
raw_data_shape = data.shape
# assuming 6 65-minute periods per day
back_day = 6*20 # 20 days
window_length = 6*250 # 250 days
train_size = 6*1000 # 1000 days

data.ffill(inplace=True)
data.bfill(inplace=True)
assert data.isna().sum().sum() == 0

data['datetime'] = pd.to_datetime(data['datetime'], utc=True)
data.set_index('datetime', inplace=True)

namelist = data.columns.tolist()

def rv(series: pd.Series, window: int) -> pd.Series:
    """
    Realized volatility is defined in [Volatility Forecasting with Machine Learning
    and Intraday Commonality](https://arxiv.org/pdf/2202.08962.pdf) as:

    $$RV_{i,t}(h)=\log(\sum_{s=t-h+1}^{t}r^2_{i,s})$$
    """
    assert window > 0, "Window must be greater than 0"
    fuzz = 1e-16
    log_returns = np.log(series).diff() # log returns
    sum_of_squares = log_returns.rolling(window=window).apply(lambda x: np.sum(x**2), raw=True)
    rv = np.log(sum_of_squares + fuzz)
    assert rv.isna().sum() == window, "RV should have NaNs at the beginning" # ? should have one nan from logret and window - 1 from rolling = window
    return rv

for ind in namelist:
    data[ind + "_logvol"] = rv(data[ind], window_length)

date = data.index

assert data.shape == (raw_data_shape[0], raw_data_shape[1]*2), "Dataframe shape is incorrect, should be the same number of rows as raw data but with double columns because we should have a price col and vol col for each ticker "

## Preprocessing

In [4]:
class Preprocess:
    # ! They put all the logic in the class constructor - i would prefer to have a separate function for each step
    def __init__(self, input, target, back_day = list(range(0,15)), forward_day = 1):
        # this liss(range(0,15)), forward_day = 1 seem to correspond to the lookback window and the forecast horizon
        # ! input is a list of dataframes, for example [price,volatility] with index as the same as target.
        # ! note that the original code in rolling_predict passes one stock at a time, so the input is a list of one dataframe
        # list of dfs holds all the results, target is the actual "input" column
        
        # ! Section 1 - make incrementally shifted seqences for each df in the input list
        self.x = []
        for df in input:
            # Shift the dataframe by each value in back_day and concatenate along columns
            # ! detailed explanation below
            shifted_df = pd.concat(
                list(map(lambda n: df.shift(n), back_day)), axis=1
            ).reset_index(drop=True).loc[:, ::-1]
            self.x.append(np.expand_dims(np.array(shifted_df), axis=2)) # Expand dimensions to make it compatible for future concatenation

        self.x = np.concatenate(tuple(self.x), axis=2) # Concatenate all processed input data along the last axis to return a 3D array

        # ! X shape = (7516, 15, 1), rows / columns / channels
        # ! X shape = (number of agg bars) / (back day list len) / (#dfs in input list)
        assert self.x.shape == (self.input[0].shape[0], len(back_day), len(input)), "Input shape is incorrect" # ! this is a sanity check to make sure the shape is correct
        
        # ! Section 2 - make the target, mask, and date
        idx1 = [~np.any(np.isnan(p)) for p in self.x] # Create an index mask where none of the elements in the x dataframes are NaN
        # ! for each row in x (which includes the row data from all dfs), if any of the values are NaN, then return False, else return True (therefore the length of idx will be the number of rows)
        self.y = target.shift(-forward_day) # Shift the target by forward_day to align with predictor variables
        self.y = pd.DataFrame(self.y).reset_index(drop=True) # Reset index to align with self.x (i removed the double parentheses around self.y)
        self.idx2 = self.y.notna().all(axis=1) # simple mask all rows where there are no NaNs (i.e. all values are present) note that this is notna, not isna like before. So this is the opposite of the previous mask
        self.idx = np.logical_and(idx1, self.idx2) # Combine the two index masks (element-wise and)
        # ! final mask "idx" is of shape (rows, )

        self.x = self.x[self.idx] # Filter x and y data based on combined index mask
        self.y = np.array(self.y[self.idx].reset_index(drop=True)) # Filter date based on combined index mask, make it an array (this changes expected dims)
        
        # ! Section 3 - make the date
        self.idx = data.index[self.idx] #! this is weird naming convention, because now self.idx is the date index from the data df, not a mask anymore

In [29]:
first_df = [data[namelist[0]+'_logvol']] # temporary list of dataframes (only one element for now)
ppd = Preprocess(first_df, data[namelist[0]+'_logvol'], list(range(0, 15))) # preprocess the data

In [5]:
def normalize_V2(x, levels=[0.01, 0.01]):
    """New windsorize function that works with 2D and 3D arrays, updated for readability"""
    # Define a function to winsorize a 1D array
    def winsorize_1d(arr):
        return mstats.winsorize(arr, levels)

    # Determine the axis along which to apply the winsorize function
    # - For 2D arrays, apply along axis 0 (columns)
    # - For 3D arrays, apply along axis 1 (rows within each 2D slice)
    axis = 1 if len(x.shape) == 3 else 0

    # Apply the winsorize function along the specified axis
    y = np.apply_along_axis(winsorize_1d, axis, x)

    return y

In [6]:
class RollingPredict:
    def __init__(self, keywords = ['XVZ_volatility'], back_day = list(range(0, 15)), lr=0.001):
        self.back_day = back_day # list of days to look back
        self.lr = lr # learning rate
        self.keywords = keywords

        first_df = [data[namelist[0]+'_logvol']] # temporary list of dataframes (only one element for now)
        self.a = Preprocess(first_df, data[namelist[0]+'_logvol'], back_day = back_day) # preprocess the data
        # ! self.a is an object with attributes x, y, and idx
        # self.a is a preprocess object
        # ! nans = number of original agg bars - window_length - back_day list length
        # ! preprocessed agg bars (ppd agg bars) = number of original agg bars - window_length - back_day list length
        # self.a.x is a numpy array of shape (6001, 15, 1) / (ppd agg bars, back day list len, #dfs in input list)
        # self.a.y is a numpy array of shape (6001, 1) / (ppd agg bars - nans, 1)
        # self.a.idx is a pandas index of timestamps of shape (6001, ) / (ppd agg bars, )

        for ind in namelist[1:]: # iterate over each ticker
            temp = [data[ind + '_logvol']] # make a list of dataframes (only one element for now) (this is passed to the input attribute of Preprocess)
            # Shape of temp = (7516, 1)
            temp_a = Preprocess(temp, data[ind + '_logvol'], back_day = back_day) 

            # ! Expected Dims for temp_a
            # temp_a.x = (6001, 15, 1) / (ppd agg bars, back day list len, #dfs in input list)

            # ! these lines essentially concat onto the first_df object stored in self.a
            # ! So theyre updating the x, y, and idx attributes of self.a
            # ! not sure why they did this
            self.a.x = np.concatenate([self.a.x, temp_a.x], axis=0) # concatenate the x data (predictor variables)
            # now (first loop) self.a.x should be of shape (12002, 15, 1) / (ppd agg bars, back day list len, #dfs in input list)
            self.a.y = np.concatenate([self.a.y, temp_a.y], axis=0) # concatenate the y data (target variable)
            # now (first loop) self.a.y should be of shape (12002, 1) / (ppd agg bars, 1)
            self.a.idx = np.concatenate([self.a.idx, temp_a.idx], axis=0) # concatenate the date data
            # now (first loop) self.a.idx should be of shape (12002, ) / (ppd agg bars, )

        ppd_agg_bars = first_df[0].shape[0] - window_length - len(back_day) # number of agg bars - window length - back day list length
        all_ppd_agg_bars = ppd_agg_bars * len(namelist) # total number of agg bars * number of tickers
        assert self.a.x.shape == (all_ppd_agg_bars, len(back_day), 1), "Preprocessed df should have dimensions (ppd agg bars * name list len, len back day list, 1)"
        assert self.a.y.shape == (all_ppd_agg_bars, 1), "Preprocessed df should have dimensions (ppd agg bars, 1)"
        assert self.a.idx.shape == (all_ppd_agg_bars, ), "Preprocessed df should have dimensions (ppd agg bars, )"

    def train(self, train_index, predict_index, lr, names, Epoch_num = 300, pre=True):

        # ! Need to figure out how all this works ========================
        temp_train_start = np.where(self.a.idx == train_index[0]) # match the date index stored in self.a.idx to the first training date
        # TODO - figure out the data structure of temp_train_start
        temp_index_train = [] # list of indices for training data

        for i in temp_train_start[0]: # for each index in temp_train_start[0]
            temp_index_train.extend(list(range(i, i + len(train_index)))) # add the indices for the training data

        temp_predict_start = np.where(self.a.idx == predict_index[0]) # get the index of the first prediction date
        temp_index_predict = []

        for i in temp_predict_start[0]:
            temp_index_predict.extend(list(range(i, i + len(predict_index)))) # add the indices for the prediction data
        
        train_x = self.a.x[temp_index_train] # get the training predictor data
        train_y = self.a.y[temp_index_train] # get the training target data
        test_x = self.a.x[temp_index_predict] # get the test predictor data
        test_y = self.a.y[temp_index_predict] # get the test target data

        # ! ===============================================================
        

In [8]:
back_day_range = list(range(0, 15))
rp = RollingPredict(back_day=back_day_range, lr=0.001)

AttributeError: 'RollingPredict' object has no attribute 'x'

In [37]:
ppd_agg_bars = data.shape[0] - window_length - len(back_day_range)
expected_shape = (ppd_agg_bars * len(namelist), len(back_day_range), 1)

In [38]:
expected_shape

(558093, 15, 1)

In [40]:
rp.a.idx.shape

(558093,)

In [24]:
rp.a.idx.shape

(558093,)

In [15]:
rp.a.idx # this is a list of timestamps from the data df (length = num rows in data df)

array([Timestamp('2019-10-15 15:40:00+0000', tz='UTC'),
       Timestamp('2019-10-15 16:45:00+0000', tz='UTC'),
       Timestamp('2019-10-15 17:50:00+0000', tz='UTC'), ...,
       Timestamp('2023-10-09 15:40:00+0000', tz='UTC'),
       Timestamp('2023-10-09 16:45:00+0000', tz='UTC'),
       Timestamp('2023-10-09 17:50:00+0000', tz='UTC')], dtype=object)

## Train function

In [ ]:
# ! deconstructing the train function
# args
train_index = None
predict_index = None

temp_train_start = np.where(rp.a.idx == train_index[0]) # match the date index stored in self.a.idx to the first training date
# TODO - figure out the data structure of temp_train_start
temp_index_train = [] # list of indices for training data

for i in temp_train_start[0]: # for each index in temp_train_start[0]
    temp_index_train.extend(list(range(i, i + len(train_index)))) # add the indices for the training data

## Run function

In [18]:
data

,TMO,ABT,HD,MCD,PG,CAT,DIS,CCI,JNJ,KO,...,COST_logvol,IBM_logvol,ADP_logvol,AVGO_logvol,WMT_logvol,AMGN_logvol,INTU_logvol,AXP_logvol,MMC_logvol,CB_logvol
datetime,,,,,,,,,,,,,,,,,,,,,
2018-10-11 13:30:00+00:00,231.4025,68.9400,195.260,165.720,80.680,143.9900,112.295,107.9300,136.72,45.3500,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-10-11 14:35:00+00:00,228.4800,68.6200,193.320,164.405,80.185,142.7600,111.960,107.0100,135.64,45.1800,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-10-11 15:40:00+00:00,231.4800,69.3857,194.630,164.970,79.970,143.6000,112.580,107.0700,136.26,45.2600,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-10-11 16:45:00+00:00,228.9800,68.8200,193.570,164.150,79.660,142.8800,112.190,106.7900,135.87,45.0800,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-10-11 17:50:00+00:00,226.8900,68.3500,190.040,162.850,79.210,141.8500,111.200,105.3590,134.41,44.7200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-09 14:35:00+00:00,492.3400,96.0800,291.235,248.350,141.960,267.2700,83.895,92.6000,157.94,52.3700,...,-3.010464,-3.320969,-3.070882,-2.343526,-3.503360,-2.974321,-2.100127,-2.440170,-3.392075,-3.049423
2023-10-09 15:40:00+00:00,492.7300,95.8850,292.350,247.935,142.340,267.8525,83.990,92.7200,157.58,52.2300,...,-3.010567,-3.321803,-3.070941,-2.343492,-3.503311,-2.973728,-2.100285,-2.440062,-3.392489,-3.050546
2023-10-09 16:45:00+00:00,494.6027,96.4200,294.245,249.410,142.935,270.1700,84.515,93.3179,158.23,52.5250,...,-3.010431,-3.321045,-3.070520,-2.342697,-3.502854,-2.973236,-2.099906,-2.440027,-3.392360,-3.050345


In [17]:
print(data.shape)
print(rp.a.x.shape)

(7516, 186)
(558093, 15, 1)


In [12]:
window_length = 6*250 # 250 days
train_size = 6*1000 # 1000 days
epoch_num = 1
pre = True

T = int(rp.a.x.shape[0]/len(namelist)) 
result_list = []

array([[[-2.74680026],
        [-2.7492632 ],
        [-2.75039409],
        ...,
        [-2.76376732],
        [-2.76364104],
        [-2.76366267]],

       [[-2.7492632 ],
        [-2.75039409],
        [-2.75210579],
        ...,
        [-2.76364104],
        [-2.76366267],
        [-2.7642805 ]],

       [[-2.75039409],
        [-2.75210579],
        [-2.75336631],
        ...,
        [-2.76366267],
        [-2.7642805 ],
        [-2.76427444]],

       ...,

       [[-3.04678476],
        [-3.04594923],
        [-3.04616489],
        ...,
        [-3.04934018],
        [-3.049423  ],
        [-3.05054623]],

       [[-3.04594923],
        [-3.04616489],
        [-3.04600895],
        ...,
        [-3.049423  ],
        [-3.05054623],
        [-3.05034485]],

       [[-3.04616489],
        [-3.04600895],
        [-3.04763715],
        ...,
        [-3.05054623],
        [-3.05034485],
        [-3.05222795]]])